In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data.txt")

In [0]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [0]:
lr_model = lr.fit(training)

In [0]:
lr_model.coefficients

Out[9]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lr_model.intercept

Out[10]: 0.14228558260358093

In [0]:
training_summary = lr_model.summary

In [0]:
training_summary.r2

Out[12]: 0.027839179518600154

In [0]:
training_summary.rootMeanSquaredError

Out[13]: 10.16309157133015

In [0]:
all_data = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data.txt")

In [0]:
train_data, test_data = split_object = all_data.randomSplit([0.7, 0.3])

Out[16]: [DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                343|
|   mean|0.48234765145440656|
| stddev| 10.332778869359553|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-30.491930077352674|
|-25.268302606922994|
| -21.42029980716747|
|-20.217891467360253|
| -19.85385587654675|
|-18.648307661032472|
|-14.669469307735625|
|-16.944848804886693|
| -17.71695105051448|
|-17.903106986170624|
|-16.331501345594997|
| -17.93124810641207|
|  -19.9024811846375|
| -17.04686998626321|
| -17.91211178372348|
| -15.91631604223563|
| -14.82030760723315|
|-13.237140725505633|
|-13.741440642598192|
|-13.945926900099833|
+-------------------+
only showing top 20 rows



In [0]:
test_results.rootMeanSquaredError

Out[23]: 10.401133654551842

In [0]:
unlabeled_data = test_data.select("features")

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   1.920451207609248|
|(10,[0,1,2,3,4,5,...|   -1.46790457567873|
|(10,[0,1,2,3,4,5,...|-0.01208795699833...|
|(10,[0,1,2,3,4,5,...|0.005814208401581422|
|(10,[0,1,2,3,4,5,...| 0.07109308693221283|
|(10,[0,1,2,3,4,5,...|  -1.019010954339247|
|(10,[0,1,2,3,4,5,...|  -2.824731049147718|
|(10,[0,1,2,3,4,5,...|-0.12055082098932413|
|(10,[0,1,2,3,4,5,...|   0.997854216909393|
|(10,[0,1,2,3,4,5,...|  1.2108999648595193|
|(10,[0,1,2,3,4,5,...| 0.24584230457350886|
|(10,[0,1,2,3,4,5,...|   2.069238778841512|
|(10,[0,1,2,3,4,5,...|   4.178965571588933|
|(10,[0,1,2,3,4,5,...|   1.671012262950915|
|(10,[0,1,2,3,4,5,...|  2.8556288091810464|
|(10,[0,1,2,3,4,5,...|  1.1535577893045041|
|(10,[0,1,2,3,4,5,...|   0.844176676080447|
|(10,[0,1,2,3,4,5,...| -0.6299471696531355|
|(10,[0,1,2,3,4,5,...| 0.32084586670743503|
|(10,[0,1,2,3,4,5,...|  0.792591